In [61]:
import pandas as pd
import numpy as np
from textblob.tokenizers import WordTokenizer as WT
from scipy.sparse import lil_matrix as lm
from textblob import TextBlob
from tqdm import tqdm
import collections
from sklearn.preprocessing import normalize
import re
pd.set_option('display.notebook_repr_html', True)


In [62]:
df = pd.read_csv("FOX.csv",sep="\t")

In [73]:
class TweetCleaner:
    
    def __init__(self, df):
        self.rawtweets = df
        self.regexReplace()

    def regexReplace(self):
        tweets = self.rawtweets.str.replace("…", "...")
        tweets = tweets[tweets]
        tweets = tweets.str.replace(r"https?\\B", "", regex=True)
        tweets = tweets.str.replace("&amp;", "&")
        tweets = tweets.str.replace("w/", "with ")
        tweets = tweets.str.replace(r"[^A-Za-z0-9:;,&\'\"\-!]", " ", regex=True)
        self.cleanedtweets = tweets.str.replace(r"^[^a-z]*:[^A-Za-z0-9]", "", regex=True)
        print(self.cleanedtweets)

TweetCleaner(df.tweet)

0         Democrats' court-packing push haunted by 2016 ...
1         Facebook censors New York Post report on BLM c...
2         Twitter accused of protecting  CNN after tech ...
3         Chicago police release Adam Toledo shooting vi...
4         NY Health Department surveyor exposes NYC nurs...
                                ...                        
387554    Transcript: Democrats' Weekly Radio Response  ...
387555    French PM Villepin Says U S  Should Pull Out o...
387556    Bush Says Congress Needs to Pass War Spending ...
387557    North Korea Says It Is Shutting Down Main Nucl...
387558    Confederate Flag Hanging From Noose Stays on D...
Name: tweet, Length: 387559, dtype: object


In [5]:
contractions = ["\'t ","\'s ","\'ve ","\'m ","\'re ","\'ll "]
fixed_contracts = ["***t ","***s ","***ve ","***m ","***re ","***ll "]

def replace_contractions(text):
    for i in range(len(contractions)):
        text = text.replace(contractions[i], fixed_contracts[i])
    return text

def fix_contractions(text):
    for i in range(len(contractions)):
        text = text.replace(fixed_contracts[i], contractions[i])
    return text

def clean(text):
    text = text.replace("’","")
    text = text.replace("‘","")
    text = text.replace("&amp;","&")
    text = text.replace("``","")
    text = text.replace("\"","")
    text = text.replace("“","")
    text = text.replace("”","")
    text = text.replace("\'", "")
    text = text.replace("…", "...")
    text = text.replace("`", "")
    text = text.replace("w/", "with ")
    return text

l = []
df["tweet"].values
for i in tqdm(df["tweet"].values):
    if ("Fox Nation" in i) or ("FoxNewsSunday"  in i) or ("Fox News Channel"  in i) or (i[:2] == ".@") or (i[:8] == "TUNE IN:") or ("Fox News Poll"  in i) or ("@" in i) or ("#" in i) or ("FOX NEWS FLASH VIDEO: " in i):
        continue
    for j in ["BUSINESS: ", "ELECTIONS: ","BREAKING: ", "UPDATE: ", "WORLD: ", "FOX LIFE: ", "POP TARTS: ", "HEALTH: ","SCIENCE: ","FOX 411: FOX 411: ", "SCITECH: ","NATIONAL: ","POLITICS: ", "FOX EMBEDS: ", "VIEWS: ", "TECHNOLOGY: ", "FOX BLOGS: REPORTER'S NOTEBOOK: Reporter's Reflections: ","FOX BLOGS: ", "FOX LIFE: FOX411: ", "FOX LIFE: ", "OPINION: ", "ENTERTAINMENT: "]: 
        if (j in i):
            i = i[len(j):]
            break
    for j in ["BUSINESS: ", "ELECTIONS: ","BREAKING: ", "UPDATE: ", "WORLD: ", "FOX LIFE: ", "POP TARTS: ", "HEALTH: ","SCIENCE: ","FOX 411: FOX 411: ", "SCITECH: ","NATIONAL: ","POLITICS: ", "FOX EMBEDS: ", "VIEWS: ", "TECHNOLOGY: ", "FOX BLOGS: REPORTER'S NOTEBOOK: Reporter's Reflections: ","FOX BLOGS: ", "FOX LIFE: FOX411: ", "FOX LIFE: ", "OPINION: ", "ENTERTAINMENT: "]: 
        if (j in i):
            i = i[len(j):]
            break
    if 'https://' in i:
        temp = replace_contractions(i)
        temp = clean(temp.split(" https://")[0])
    elif 'http://' in i:
        temp = replace_contractions(i)
        temp = clean(temp.split(" http://")[0])
    else:
        temp = replace_contractions(i)
        temp = clean(temp)
    temp = fix_contractions(temp)
    temp = temp.strip()
    
    if temp[-3:] == "...":
        temp = temp.split(".")[0]
    
    l += [temp]

l = list(set(l))

l, len(l)

100%|███████████████████████████████| 387559/387559 [00:02<00:00, 188441.69it/s]


(['',
  "Mueller's team slams Manafort's lavish lifestyle in the trial's opening statements, as the defense says wealth is no crime.",
  'The Hartford To Sell Stock, Take Government Cash',
  'Further Crackdowns on Freedom in Russia, Despite Promises',
  'Paccar Earnings Slump To 4 Cents a Share. Paccar Earnings Slump To 4 Cents a Share',
  'Apple Announces OS X Snow Leopard Release Date',
  'Boy Saved From Rabid Mountain Lion',
  'Google to Let Publishers Limit Free Content',
  'Iran planned terror attacks against Israel advocates in Germany',
  'Happening Now: President Donald Trump delivers remarks from MacDill Air Force Base in Florida.',
  'McCain Will Attend Friday Debate',
  'Intl Game Technology 1st-quarter Net Off 6%, Revenue Up 0.5%',
  "Kori DeLuca on cruise ship hit by high winds: As the hrs went by & it just didn't get better...after the tenth hour my nerves were shot.",
  'Treasurys Fall Ahead Of Housing Data, Bernanke',
  'Boeing CEO: No Headway In Strike Negotiations',
 

In [29]:
df.tweet

0         Democrats' court-packing push haunted by 2016 ...
1         Facebook censors New York Post report on BLM c...
2         Twitter accused of protecting @CNN after tech ...
3         Chicago police release Adam Toledo shooting vi...
4         NY Health Department surveyor exposes NYC nurs...
                                ...                        
387554    POLITICS: Transcript: Democrats' Weekly Radio ...
387555    WORLD: French PM Villepin Says U.S. Should Pul...
387556    POLITICS: Bush Says Congress Needs to Pass War...
387557    WORLD: North Korea Says It Is Shutting Down Ma...
387558    NATIONAL: Confederate Flag Hanging From Noose ...
Name: tweet, Length: 387559, dtype: object

In [9]:
cleaned = []
extras = []
punctuation = {":", ";", ",", ".", "?", "!", "$", "(", ")", "..."}
tok = WT()
for p in tqdm(range(len(l))):
    text = l[p]
    if text == "":
        continue
        
    if text[-7:-1] == "------":
        text = text[:-7] + "ull!"
    if (text[-1:] == "-"):
        text = text[:-1]
    if (text[-2:] == "--"):
        text = text[:-2]
    if (text[-3:] == "---"):
        text = text[:-3]
    if text[-1] == "-":
        text = text[:-1]
    while "::" in text:
        text = text.replace("::", ":")
    
    text = text.strip()
    
    if text[-1] not in punctuation:
        text = text + "."
    
    blob = TextBlob(text)
    split = tok.tokenize(text)
    tags = blob.tags
    j = 0
    for i in range(len(split)):
        if j >= len(tags) and (split[i] in punctuation):
            extras += [split[i]]
            split[i] = (split[i], split[i])
            continue
        if split[i] != tags[j][0]:
            split[i] = (split[i], split[i])
        else:
            split[i] = tags[j]
            j += 1
    cleaned += [split]

100%|█████████████████████████████████| 220354/220354 [02:36<00:00, 1412.50it/s]


In [10]:
use_cases = []
li = cleaned
new_l = []
strucs = []
for i in tqdm(li):
    inner = [("", "$tart")]
    for j in range(len(i)):
        if j == 0:
            word, use = i[j]
            use_cases += [use]
            continue
        new_word, new_use = i[j]
        use_cases += [new_use]
        if use == new_use:
            word = word + " " + new_word
        else:
            inner += [(word,use)]
            word, use = new_word, new_use
        if j == len(i) -1:
            if new_use != use:
                inner += [(new_word, new_use)]
            else:
                inner += [(word, use)]
    if inner[-1][1] == ".":
        word = inner[-1][0]
        inner[-1] = (word, "$top")
    else:
        inner += [("", "$top")]
    new_l += [inner]

100%|███████████████████████████████| 220353/220353 [00:02<00:00, 102930.68it/s]


In [11]:
pairs = []
words = set()
for i in new_l:
    for j in range(1,len(i)):
        a, b = i[j-1]
        c, d = i[j]
        word_use_1 = a+"*^*"+b
        word_use_2 = c+"*^*"+d
        pairs += [(word_use_1, word_use_2)]
        words.add(word_use_1)
        words.add(word_use_2)

        
counter = collections.Counter(pairs)
counter.most_common(), len(set(pairs))

([(('%*^*NN', '.*^*$top'), 2725),
  (('?*^*?', '*^*$top'), 2706),
  (('say*^*VBP', '.*^*$top'), 2405),
  (('says*^*VBZ', '.*^*$top'), 2364),
  (('Report*^*NN', ':*^*:'), 2336),
  (('*^*$tart', 'Report*^*NN'), 2308),
  ((':*^*:', '*^*$top'), 2207),
  (('of*^*IN', 'the*^*DT'), 2032),
  (('in*^*IN', 'the*^*DT'), 1883),
  (('*^*$tart', 'Obama*^*NNP'), 1495),
  (('*^*$tart', 'The*^*DT'), 1397),
  ((':*^*:', 'The*^*DT'), 1380),
  (('to*^*TO', 'be*^*VB'), 1321),
  (('*^*$tart', 'Man*^*NN'), 1286),
  ((':*^*:', 'I*^*PRP'), 1230),
  (('4*^*CD', ':*^*:'), 1180),
  (('on*^*IN', 'the*^*DT'), 1179),
  (('*^*$tart', 'ROW*^*NNP'), 1172),
  (('ROW*^*NNP', '2*^*CD'), 1172),
  (('2*^*CD', 'SEAT*^*NNP'), 1172),
  (('SEAT*^*NNP', '4*^*CD'), 1172),
  (('to*^*TO', 'the*^*DT'), 1102),
  (('*^*$tart', 'Trump*^*NNP'), 1076),
  (('police*^*NNS', 'say*^*VBP'), 1057),
  ((':*^*:', 'We*^*PRP'), 1009),
  ((':*^*:', 'Report*^*NN'), 952),
  ((',*^*,', 'police*^*NNS'), 921),
  (('*^*$tart', 'A*^*DT'), 913),
  (('*^*$t

In [12]:
words = set()
for i in pairs:
    a, b = i
    if a not in words:
        words.add(a)
    if b not in words:
        words.add(b)
num_words = len(words)

In [15]:
indexed = set()
get_index = dict()
m = len(set(pairs))
f_mat = lm((num_words,num_words))
pair_freq = counter.most_common()

ind = 0
for i in pair_freq:
    pair, freq = i
    l, r = pair
    if l not in indexed:
        get_index[l] = ind
        indexed.add(l)
        ind += 1
    if r not in indexed:
        get_index[r] = ind
        ind += 1
        indexed.add(r)
    f_mat[get_index[l], get_index[r]] = freq
    
f_mat = normalize(f_mat, norm='l1', axis=1)
rev_map = {v: k for k, v in get_index.items()}

In [22]:
ind = get_index["*^*$tart"]


for i in range(10):
    sent = []
    ind = 10
    while 1:
        new_ind = np.random.choice(np.arange(f_mat[0].shape[1]), p=f_mat.getrow(ind).toarray()[0])
        new_word = rev_map[new_ind]
        ind = new_ind
        sent += [new_word.split("*^*")[0]]
        if new_word[-4:] == "$top":
            break

    print(" ".join(sent))
    print('\n')

rule .


Racecourse , son dies at California dam increase , gang terrorizing our congressionally assigned to rejoin Breitbart reporter .


Brooklyn Bridge to celebrate than just in Spain , report says it an extremely disappointed over immigration clause in Front Seat .


day in building would n't have invaded Crimea , reports shock New Orleans shootings taken off food guideline to End Up to Die : his life .


communities .


City Stars Confirmed for your bank account .


rule of terror | .


World Now in same small town allegedly left to be sleazy but he fired , China Mobile 2008 Chevrolet Malibu .


latest insurer to be hit man in formula to Bring Them Home .


Red : If eating ice cream coolers from home at Least 22 Mln In Tax-cut Squabble .




756

In [207]:
f_mat[10].todense()

[matrix([[0., 0., 0., ..., 0., 0., 0.]])]

array([0., 0., 0., ..., 0., 0., 0.])